In [2]:
import pandas as pd
from clickhouse_driver import Client
import json

In [3]:
# данные для подлкючения вынесены в отдельный файл
with open('entities.json', encoding='utf-8') as f:
    templates = json.load(f)

host = templates['host']
user = templates['user']
password = templates['password']
database = templates['database']

In [35]:
#Подключаемся к БД и проверяем наличие необзодимой таблицы
client = Client(host,
                user=user,
                password=password,
                database=database)
client.execute('DESCRIBE TABLE marvel')


[('user_id', 'UUID', '', '', '', '', ''),
 ('installed_at', 'DateTime', '', '', '', '', ''),
 ('created_at', 'DateTime', '', '', '', '', ''),
 ('day', 'Int32', '', '', '', '', ''),
 ('country', 'LowCardinality(String)', '', '', '', '', ''),
 ('app_version_short', 'LowCardinality(String)', '', '', '', '', ''),
 ('last_time_spent', 'UInt32', '', '', '', '', ''),
 ('lifetime_session_count', 'UInt32', '', '', '', '', ''),
 ('activity_kind', 'LowCardinality(String)', '', '', '', '', ''),
 ('event_name', 'LowCardinality(String)', '', '', '', '', ''),
 ('reporting_revenue', 'Float32', '', '', '', '', '')]

In [36]:
#Есть несколько способов: 
#написать сложные sql запросы и потом в питоне данные обработать немного
#написать простой sql запрос и обработать данные в питоне необходимым способом
#В данном случае в питоне можно довольно быстро поработать с данными + он мне удобен, поэтому я выбираю данный подход

rr_data = client.execute('''
                                select 
                                    user_id,
                                    formatDateTime(installed_at, '%F') AS installed_at,
                                    day AS day_after_install
                                from marvel
                                group by user_id, installed_at, day_after_install
                             ''')
#группировка, чтобы убрать дубли лишние

In [37]:
rr_data_df = pd.DataFrame(rr_data, columns=('uuid', 'installed_at', 'day_after_install'))

In [38]:
# Обработает дату, установки
rr_data_df['installed_at'] = pd.to_datetime(rr_data_df['installed_at'], format='%Y-%m-%d')
metrics_data_df.head()

,uuid,created_at_dt,installed_at,day_after_install,revenue,year_created_at,month_created_at,week_created_at,year_installed_at,month_installed_at,week_installed_at
0,626521f3-8eeb-ccee-6428-1c8883c09623,2020-05-09,2020-05-05,4,0.0,2020,5,19,2020,5,19
1,34c9c565-314b-df4c-2e1c-cae6fbd190e1,2020-06-27,2020-06-25,1,0.0,2020,6,26,2020,6,26
2,b4cd8fc9-a73d-33bb-9958-095517b7c67e,2020-04-13,2020-04-13,0,0.0,2020,4,16,2020,4,16
3,8c4b0dd5-71da-b261-2176-0a36b79b9f60,2020-03-30,2020-03-30,0,0.0,2020,3,14,2020,3,14
4,ef7c8ba1-7378-9201-7df9-70c16d55cb7f,2020-05-28,2020-05-21,6,0.0,2020,5,22,2020,5,21


In [39]:
#Формуруем датасет для визуализации RR
new_users = metrics_data_df.groupby('installed_at').uuid.nunique().reset_index()
retained_users = metrics_data_df[metrics_data_df['day_after_install'] > 0].groupby(
                                                        ['installed_at','day_after_install']).uuid.nunique().reset_index()

retained_users.rename(columns={'installed_at': 'dt', 'uuid': 'retained_users_count'}, inplace=True)
new_users.rename(columns={'installed_at': 'dt', 'uuid': 'new_users_count'}, inplace=True)

rr_df = new_users.merge(retained_users, how='outer', on='dt')
rr_df['retention'] = rr_df['retained_users_count'] / rr_df['new_users_count']

In [40]:
rr_df.head()

,dt,new_users_count,day_after_install,retained_users_count,retention
0,2019-12-16,2,32.0,1.0,0.5
1,2019-12-16,2,43.0,1.0,0.5
2,2019-12-16,2,59.0,1.0,0.5
3,2019-12-16,2,60.0,1.0,0.5
4,2019-12-16,2,62.0,1.0,0.5


In [20]:
arpu = client.execute('''
                                select 
                                    created_at_dt,
                                    countIf(revenue == 0) AS users_count_without_revenue,
                                    countIf(revenue > 0) AS users_count_with_revenue,
                                    sum(revenue)
                                from (
                                    select 
                                        formatDateTime(created_at, '%F') AS created_at_dt,
                                        user_id,
                                        sum(reporting_revenue) AS revenue
                                    from marvel
                                    group by user_id, created_at_dt
                                    )
                                group by created_at_dt
                                order by created_at_dt asc
                             ''')

In [43]:
arpu_df = pd.DataFrame(arpu, columns=('dt', 'users_count_without_revenue', 'users_count_with_revenue', 'revenue'))

In [44]:
arpu_df['arpu'] = arpu_df['revenue'] / (arpu_df['users_count_without_revenue'] + arpu_df['users_count_with_revenue'])

arpu_df['dt'] = pd.to_datetime(arpu_df['dt'], format='%Y-%m-%d') 
arpu_df.head()

,dt,users_count_without_revenue,users_count_with_revenue,revenue,arpu
0,2020-01-17,0,1,143.945755,143.945755
1,2020-01-20,0,2,10.810568,5.405284
2,2020-01-21,0,1,6.488145,6.488145
3,2020-01-22,0,1,6.675666,6.675666
4,2020-01-23,0,13,996.697218,76.669017


In [45]:
arpu_df

,dt,users_count_without_revenue,users_count_with_revenue,revenue,arpu
0,2020-01-17,0,1,143.945755,143.945755
1,2020-01-20,0,2,10.810568,5.405284
2,2020-01-21,0,1,6.488145,6.488145
3,2020-01-22,0,1,6.675666,6.675666
4,2020-01-23,0,13,996.697218,76.669017
5,2020-01-24,0,34,801.409727,23.570874
6,2020-01-25,0,27,292.450712,10.831508
7,2020-01-26,0,32,1409.428600,44.044644
8,2020-01-27,0,50,7103.790669,142.075813
9,2020-01-28,0,59,7813.744248,132.436343


In [46]:
#выгрузим данные в csv для загрузки в Tableu
rr_df.to_csv('AQ_output_rr.csv')
arpu_df.to_csv('AQ_output_arpu.csv')